<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2112fff239641132fa999660e2da445381d645ec4dc7e9dcc902f1f9f2d2c1e6
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-09 12:58:40
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 50.22 K (0.36%)
Current PnL: -20.75 L (-13.77%)
CY Booked + Current PnL: -9.31 L (-6.18%)
-------------------
Total profit:  1.59 L
Total loss:  -22.34 L
-------------------
Total Booked + Current PnL: 17.67 L (14.28%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.12%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.26 L (62.67%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121184.0,20680.0,8422.0,2.25,20.58,6.95,28.96,79.0,2.46,0.86,61.53,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.38,40.0,M-SC,1.00,83355.0,-17422.0,17513.0,0.16,-17.29,21.01,0.09,245.0,-0.99,0.59,9.51,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.02,46.0,H-MC,3.03,188270.0,23600.0,19731.0,-0.51,14.33,10.48,26.31,89.0,1.20,1.34,23.46,X40N,NTT,AC
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.67,207336.0,2836.0,20464.0,1.26,1.39,9.87,11.39,10.0,0.14,1.48,10.46,X40N,ATH,PHARMA
84,VOLTAS,1530.00,-0.94,54.0,H-MC,1.18,207900.0,16158.0,21601.0,1.37,8.43,10.39,19.69,99.0,0.75,1.48,15.80,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,1006.00,-40.66,48.0,M-SC,13.74,73868.0,-27291.0,102182.0,4.16,-26.98,138.33,74.03,202.0,-0.27,0.53,4.16,AR,BTT,AUTO
55,RAJOOENG,143.10,-40.28,46.0,H-SC,16.39,90450.0,-12050.0,52651.0,4.09,-11.76,58.21,39.61,114.0,-0.23,0.64,10.47,AR,ATH,MISC
32,HINDZINC,730.22,35.55,73.0,M-LC,7.33,220588.0,15512.0,97059.0,3.38,7.56,44.00,54.89,52.0,0.16,1.57,33.48,X5K,ATH,METALS
21,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.38,80640.0,-32909.0,72963.0,3.27,-28.98,90.48,35.27,268.0,-0.45,0.57,97.37,XR,NTT,HOTELS
40,INFY,2275.00,-18.16,54.0,H-LC,8.00,318435.0,5547.0,166128.0,2.50,1.77,52.17,54.87,3.0,0.03,2.27,8.54,X40,BTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CERA,9475.00,-23.87,36.0,H-SC,1.69,138690.0,-37213.0,79234.0,-2.38,-21.16,57.13,23.89,149.0,-0.47,0.99,19.78,OX40N,NTT,CERAMICS
12,BATAINDIA,2096.00,-34.87,43.0,M-SC,4.40,97351.0,-31319.0,76615.0,-2.34,-24.34,78.70,35.20,219.0,-0.41,0.69,11.74,X40,NTT,FOOTWEAR
58,REPCOHOME,880.00,-59.02,57.0,H-SC,2.76,235489.0,-51016.0,308349.0,-1.88,-17.81,130.94,89.82,134.0,-0.17,1.68,24.59,XY24,NTT,FINANCE
52,PGHH,17907.65,-27.24,57.0,H-MC,2.04,210435.0,9615.0,58185.0,-1.81,4.79,27.65,33.76,80.0,0.17,1.50,10.23,X40,ATH,FMCG
7,ATULAUTO,844.00,3770.27,57.0,M-SC,6.29,173842.0,-23290.0,113119.0,-1.64,-11.81,65.07,45.57,236.0,-0.21,1.24,23.25,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,61.0,H-SC,13.53,130113.0,-12366.0,31891.0,1.03,-8.68,24.51,13.70,163.0,-0.39,0.93,53.57,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,40.0,M-SC,1.00,83355.0,-17422.0,17513.0,0.16,-17.29,21.01,0.09,245.0,-0.99,0.59,9.51,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,47.0,H-SC,1.97,220581.0,-49086.0,85409.0,0.20,-18.20,38.72,13.47,138.0,-0.57,1.57,12.16,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-22.37,53.0,H-MC,6.53,105108.0,-26727.0,68951.0,1.32,-20.27,65.60,32.03,98.0,-0.39,0.75,17.73,OX40N,NTT,IT
18,CERA,9475.0,-23.87,36.0,H-SC,1.69,138690.0,-37213.0,79234.0,-2.38,-21.16,57.13,23.89,149.0,-0.47,0.99,19.78,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121184.0,20680.0,8422.0,2.25,20.58,6.95,28.96,79.0,2.46,0.86,61.53,MH,ATH,METALS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,46.75,56.0,M-MC,12.57,231178.0,6216.0,161432.0,1.92,2.76,69.83,74.53,192.0,0.04,1.65,35.91,XY24,BTT,STEEL
32,HINDZINC,730.22,35.55,73.0,M-LC,7.33,220588.0,15512.0,97059.0,3.38,7.56,44.00,54.89,52.0,0.16,1.57,33.48,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.02,46.0,H-MC,3.03,188270.0,23600.0,19731.0,-0.51,14.33,10.48,26.31,89.0,1.20,1.34,23.46,X40N,NTT,AC
84,VOLTAS,1530.00,-0.94,54.0,H-MC,1.18,207900.0,16158.0,21601.0,1.37,8.43,10.39,19.69,99.0,0.75,1.48,15.80,XY25,NTT,AC
17,CAMS,5211.76,-6.27,46.0,H-SC,1.65,106680.0,4676.0,39248.0,0.79,4.58,36.79,43.06,122.0,0.12,0.76,22.48,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121184.0,20680.0,8422.0,2.25,20.58,6.95,28.96,79.0,2.46,0.86,61.53,MH,ATH,METALS
32,HINDZINC,730.22,35.55,73.0,M-LC,7.33,220588.0,15512.0,97059.0,3.38,7.56,44.00,54.89,52.0,0.16,1.57,33.48,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.50,193234.0,13426.0,98994.0,0.65,7.47,51.23,62.52,88.0,0.14,1.38,44.10,XR,NTT,BANKS
37,INDIAMART,4810.62,-54.08,45.0,H-SC,8.95,128032.0,4696.0,126931.0,-0.90,3.81,99.14,106.72,119.0,0.04,0.91,28.08,AR,ATH,MISC
86,WIPRO,420.00,-12.89,51.0,M-LC,5.65,152824.0,1879.0,107573.0,0.91,1.24,70.39,72.51,53.0,0.02,1.09,7.34,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.68,31.0,H-LC,5.63,205260.0,-46508.0,99715.0,0.17,-18.47,48.58,21.13,27.0,-0.47,1.46,10.41,X40,ATH,PAINTS
83,VBL,671.64,-22.45,32.0,H-LC,10.36,279540.0,-36302.0,148967.0,0.76,-11.49,53.29,35.67,5.0,-0.24,1.99,0.77,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,39.0,H-LC,7.53,245280.0,-16339.0,135542.0,-0.36,-6.25,55.26,45.57,7.0,-0.12,1.75,4.37,AR,NTT,ELECTRICAL
42,ITC,452.00,-39.61,40.0,H-LC,2.27,195502.0,-4636.0,25533.0,0.01,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.34,43.0,H-LC,8.02,251870.0,-5193.0,40324.0,0.74,-2.02,16.01,13.67,24.0,-0.13,1.80,17.98,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-1.11,49.0,H-LC,1.94,158865.0,-27230.0,74714.0,-1.23,-14.63,47.03,25.51,15.0,-0.36,1.13,17.07,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,40.0,H-LC,2.27,195502.0,-4636.0,25533.0,0.01,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
48,LTIM,7230.2,-2.02,61.0,H-LC,2.55,242730.0,-3421.0,82625.0,0.96,-1.39,34.04,32.18,16.0,-0.04,1.73,34.15,X200,ATH,IT
51,NESTLEIND,1377.0,-10.26,51.0,H-LC,2.61,276190.0,10764.0,46041.0,-0.02,4.06,16.67,21.40,11.0,0.23,1.97,10.97,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.41,47.0,H-LC,3.08,214375.0,4009.0,24782.0,0.50,1.91,11.56,13.68,37.0,0.16,1.53,18.74,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.45,32.0,H-LC,10.36,279540.0,-36302.0,148967.0,0.76,-11.49,53.29,35.67,5.0,-0.24,1.99,0.77,X40N,ATH,FMCG
42,ITC,452.00,-39.61,40.0,H-LC,2.27,195502.0,-4636.0,25533.0,0.01,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
1,ABB,7934.00,-41.33,39.0,H-LC,7.53,245280.0,-16339.0,135542.0,-0.36,-6.25,55.26,45.57,7.0,-0.12,1.75,4.37,AR,NTT,ELECTRICAL
80,UNITDSPR,1644.00,-12.09,52.0,H-LC,6.66,230480.0,-3664.0,52296.0,0.52,-1.56,22.69,20.77,86.0,-0.07,1.64,5.24,X40N,NTT,BREWERIES
74,TCS,4406.34,-28.04,53.0,H-LC,12.15,286662.0,-59294.0,127536.0,0.74,-17.14,44.49,19.73,1.0,-0.46,2.04,5.58,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-1.11,49.0,H-LC,1.94,158865.0,-27230.0,74714.0,-1.23,-14.63,47.03,25.51,15.0,-0.36,1.13,17.07,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,40.0,H-LC,2.27,195502.0,-4636.0,25533.0,0.01,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.68,31.0,H-LC,5.63,205260.0,-46508.0,99715.0,0.17,-18.47,48.58,21.13,27.0,-0.47,1.46,10.41,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.67,207336.0,2836.0,20464.0,1.26,1.39,9.87,11.39,10.0,0.14,1.48,10.46,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.41,47.0,H-LC,3.08,214375.0,4009.0,24782.0,0.50,1.91,11.56,13.68,37.0,0.16,1.53,18.74,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.04,53.0,H-LC,12.15,286662.0,-59294.0,127536.0,0.74,-17.14,44.49,19.73,1.0,-0.46,2.04,5.58,X40,ATH,IT
40,INFY,2275.00,-18.16,54.0,H-LC,8.00,318435.0,5547.0,166128.0,2.50,1.77,52.17,54.87,3.0,0.03,2.27,8.54,X40,BTT,IT
42,ITC,452.00,-39.61,40.0,H-LC,2.27,195502.0,-4636.0,25533.0,0.01,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
83,VBL,671.64,-22.45,32.0,H-LC,10.36,279540.0,-36302.0,148967.0,0.76,-11.49,53.29,35.67,5.0,-0.24,1.99,0.77,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,39.0,H-LC,7.53,245280.0,-16339.0,135542.0,-0.36,-6.25,55.26,45.57,7.0,-0.12,1.75,4.37,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.38,80640.0,-32909.0,72963.0,3.27,-28.98,90.48,35.27,268.0,-0.45,0.57,97.37,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,41.0,H-SC,10.61,90405.0,-7575.0,29174.0,-0.87,-7.73,32.27,22.05,152.0,-0.26,0.64,31.41,XR,ATH,FINANCE
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121184.0,20680.0,8422.0,2.25,20.58,6.95,28.96,79.0,2.46,0.86,61.53,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,61.0,H-SC,13.53,130113.0,-12366.0,31891.0,1.03,-8.68,24.51,13.70,163.0,-0.39,0.93,53.57,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.40,145440.0,-3780.0,147956.0,1.58,-2.53,101.73,96.62,208.0,-0.03,1.04,57.40,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.40,145440.0,-3780.0,147956.0,1.58,-2.53,101.73,96.62,208.0,-0.03,1.04,57.40,XY25,NTT,FINANCE
32,HINDZINC,730.22,35.55,73.0,M-LC,7.33,220588.0,15512.0,97059.0,3.38,7.56,44.00,54.89,52.0,0.16,1.57,33.48,X5K,ATH,METALS
69,SURYODAY,240.00,64.47,63.0,H-SC,8.06,149940.0,-29131.0,94867.0,1.66,-16.27,63.27,36.71,135.0,-0.31,1.07,48.77,XR,NTT,BANKS
26,GREENPANEL,537.00,233.98,60.0,M-SC,4.91,149182.0,-33896.0,112871.0,-0.50,-18.51,75.66,43.14,230.0,-0.30,1.06,40.33,XY24,NTT,MISC
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121184.0,20680.0,8422.0,2.25,20.58,6.95,28.96,79.0,2.46,0.86,61.53,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.13
1,25,43.46
2,50,75.16


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.95
LC    32.27
MC    23.74
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.94
X40      14.63
X40N     11.18
XR       10.93
XY25     10.41
AR        8.16
OX40N     7.68
X200      1.73
MH        1.61
X5K       1.57
SR        1.12
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.94
H-LC    25.74
H-MC    20.46
M-SC    13.59
M-LC     5.52
M-MC     2.98
L-SC     1.42
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.71,-6.74,41.92
IT,12.89,-16.28,79.26
FINANCE,10.49,-14.75,70.17
MISC,6.92,-18.17,83.65
BANKS,6.35,-12.22,71.25
PAINTS,5.50,-19.70,37.54
ELECTRICAL,5.31,-10.40,49.14
INSURANCE,3.82,-3.29,41.90
AC,3.51,1.83,14.60


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3233690.0,22
XR,1375572.0,14
AR,1185741.0,9
X40,785898.0,10
X40N,665801.0,9
OX40N,569509.0,10
XY25,478426.0,7
SR,264816.0,2
X5K,97059.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3733684.0,29
M-SC,1521965.0,17
H-MC,1284081.0,15
H-LC,1183458.0,15
M-LC,375214.0,4
M-MC,364867.0,2
L-SC,260699.0,3
L-MC,60019.0,1
L-LC,42005.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1206721.0      6
M-SC       XY24       849399.0      7
H-SC       AR         807657.0      5
           XR         803993.0      7
H-MC       XY24       573719.0      4
H-LC       X40        489301.0      5
M-MC       XY24       364867.0      2
H-SC       X40N       307379.0      4
           SR         264816.0      2
           OX40N      264685.0      4
H-LC       X40N       221727.0      3
H-MC       X40        219982.0      4
H-LC       AR         210256.0      2
H-MC       XY25       177318.0      2
L-SC       XR         171685.0      2
M-LC       XY24       170582.0      2
M-SC       AR         167828.0      2
           XY25       147956.0      1
           OX40N      146859.0      4
H-MC       X40N       136695.0      2
M-SC       XR         133308.0      2
H-LC       XY25       111147.0      3
M-LC       XR         107573.0      1
H-MC       XR          98994.0      1
M-LC       X5K         97059.0      1
L-SC       OX40N       89014.0      1
H-LC       X200        82625.0      1
H-SC       MH          78433.0      1
M-SC       X40         76615.0      1
H-MC       OX40N       68951.0      1
H-LC       XY24        68402.0      1
L-MC       XR          60019.0      1
L-LC       XY25        42005.0      1
H-MC       MH           8422.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
